<a href="https://colab.research.google.com/github/mlarunreddy/chennai-house-price-pridiction/blob/main/Copy_of_house_predicton_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The aim of the project is to help customer predict home prices in chennai**

Importing Required Pacages

In [ ]:
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor
import numpy as np
import matplotlib.pyplot as plt 
import io
from google.colab import files 

Loding The Dataset

In [ ]:
uploaded_file=files.upload()


In [ ]:
df=pd.read_csv(io.BytesIO(uploaded_file['train chennai sale.csv']))

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

Checking Missing Values

In [ ]:
df.isnull().sum()

**Data Cleaning**

In [ ]:
df.drop_duplicates().shape

Missing Value Handeling

In [ ]:
df['N_BATHROOM'].fillna(df['N_BATHROOM'].mode()[0],inplace=True)
df['N_BEDROOM'].fillna(df['N_BEDROOM'].mode()[0],inplace=True)
df.QS_OVERALL = df.QS_OVERALL.fillna(df.QS_OVERALL.mean())

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
df['UTILITY_AVAIL'].unique()

In [ ]:
nf=df.drop(['PRT_ID','REG_FEE','COMMIS'],axis=1)

In [ ]:
nf.head()

Checking Data Types

In [ ]:
nf.dtypes

Changing To Proper Datatype

In [ ]:
nf['N_BEDROOM']=nf['N_BEDROOM'].astype(int)
nf['N_BATHROOM']=nf['N_BATHROOM'].astype(int)

In [ ]:
nf.dtypes

In [ ]:
df=nf

Creating Age of the building column based on the date sale and datebuild columns

In [ ]:
df['AGE']=pd.DatetimeIndex(df['DATE_SALE']).year-pd.DatetimeIndex(df['DATE_BUILD']).year

In [ ]:
df.dtypes

All the columns are changed to proper Datatype

In [ ]:
df.head()

Describing the Numerical Value

In [ ]:
df.describe()

checking the Outliers

In [ ]:
sns.boxplot(df['SALES_PRICE'])

In [ ]:
nf.loc[nf['STREET']=='Pavad','STREET']='Paved'
nf.loc[nf['STREET']=='NOAccsess','STREET']='No Access'
nf["STREET"].unique()

In [ ]:
st_s=nf.groupby('STREET')['SALES_PRICE'].mean()
st_s

In [ ]:
st_s.plot(kind='bar',title='House Price in STREET',ylabel='Mean Price',
          xlabel='Street',figsize=(6,5))

Findind  IQR

In [ ]:
percentile25 =df['SALES_PRICE'].quantile(0.25)
percentile75 = df['SALES_PRICE'].quantile(0.75)
print("25th percentile of the data is",percentile25)
print("75th percentile of the data is ",percentile75)
print("IQR:",percentile75-percentile25)

Calculating the MIN and MAX Limits

In [ ]:
max_value=percentile75 + 1.5 * (percentile75-percentile25)
min_value=percentile25 - 1.5*(percentile75-percentile25)
print(max_value)
print(min_value)

Removing Outliers by using Capping Technique

In [ ]:
df['SALES_PRICE']=np.where(
    df['SALES_PRICE']>max_value,
    max_value,
    np.where(
        df['SALES_PRICE']<min_value,
        min_value,
        df['SALES_PRICE']
                               
    )   
)
df['SALES_PRICE'].describe()


In [ ]:
sns.boxplot(df['SALES_PRICE'])

In [ ]:
print(df['AREA'].unique())
print(df['SALES_PRICE'].unique())
print(df['PARK_FACIL'].unique())
print(df['BUILDTYPE'].unique())
print(df['UTILITY_AVAIL'].unique())
print(df['STREET'].unique())
print(df['MZZONE'].unique())

In [ ]:
 df['AREA'].replace(['Karapakam'],'Karapakkam',inplace=True)
 df['AREA'].replace(['Ana Nagar','Ana Nagar','Ann Nagar'],'Anna Nagar',inplace=True)
 df['AREA'].replace(['Adyr'],'Adyar',inplace=True)
 df['AREA'].replace(['Velchery'],'Velachery',inplace=True)
 df['AREA'].replace(['Chrompet','Chrmpet','Chrompt'],'Chrompet',inplace=True)
 df['AREA'].replace(['KKNagar'],'KK Nagar',inplace=True)
 df['AREA'].replace(['TNagar'],'T Nagar',inplace=True)

 df['SALE_COND'].replace(['Ab Normal'],'Abnormal',inplace=True)
 df['SALE_COND'].replace(['PartiaLl','Partiall'],'Partial',inplace=True)
 df['SALE_COND'].replace(['Adj Land'],'Adjland',inplace=True)

 df['PARK_FACIL'].replace(['Noo'],'No',inplace=True)

 df['BUILDTYPE'].replace(['Others'],'Other',inplace=True)
 df['BUILDTYPE'].replace(['Comercial','commercial'],'commercial',inplace=True)
 
 df['UTILITY_AVAIL'].replace(['AllPub'],'All Pub',inplace=True)
 df['UTILITY_AVAIL'].replace(['NoSewr','NoSeWa'],'NoSewr',inplace=True)

df['STREET'].replace(['Pavd'],'Paved',inplace=True)
df['STREET'].replace(['NoAccess'],'No Access',inplace=True)

In [ ]:
print(df['AREA'].unique())
print(df['SALES_PRICE'].unique())
print(df['PARK_FACIL'].unique())
print(df['BUILDTYPE'].unique())
print(df['UTILITY_AVAIL'].unique())
print(df['STREET'].unique())
print(df['MZZONE'].unique())

After correcting all spelling mistakes in catagorical column,the above are the unique names.

In [ ]:
df.columns

Exploteory data analasis

EDA for Catagorical Columns

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df.corr(),annot=True,linewidth=0.2)

SALES_PRICE is the target.so some features have strong co-relation with respect to sales price those are INT_SQFT,N_BEDROOM,N_BATHROOM,N_ROOMS.

Some features have weekest co-relation those are QS_ROOMS,QS_BATHROOM,QS_BEDROOM,QS_ROOMS,QS_OVERALL

*Checking* relationship between the AREA feature and SALES_PRICE Target columns

In [ ]:
area_order=df.groupby('AREA')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['AREA']

In [ ]:
sns.barplot(x='AREA',y='SALES_PRICE',data=df,order=area_order)
plt.xticks(rotation=45) 

Anna Nagar and T Nagar are the area's with highest sale-price


In [ ]:
df.groupby('AREA')['SALES_PRICE'].mean().plot()
plt.title('AREA vs SALES_PRICE')
plt.xlabel('AREA')
plt.ylabel('SALES_PRICE')
plt.xticks(rotation=45)
plt.show()

Checking Relationship between SALES_COND and SALES_PRICE

In [ ]:
sale_cond_order=df.groupby('SALE_COND')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['SALE_COND']

In [ ]:
sns.barplot(x='SALE_COND',y='SALES_PRICE',data=df,order=sale_cond_order)

In SALE_COND there are slight variation

In [ ]:
df.groupby('SALE_COND')['SALES_PRICE'].mean().plot()
plt.title('SALE_COND vs SALES_PRICE')
plt.xlabel('SALE_COND')
plt.ylabel('SALES_PRICE')
plt.xticks(rotation=45)
plt.show()

Checking Relationship between PARK_FACIL and SALES_PRICE

In [ ]:
sns.barplot(x="PARK_FACIL",y='SALES_PRICE',data=df)



*Houses having Park Facility are more expencive than the house with no park facility



In [ ]:
df.groupby('PARK_FACIL')['SALES_PRICE'].mean().plot()
plt.title('PARK_FACIL vs SALES_PRICE')
plt.xlabel('PARK_FACIL')
plt.ylabel('SALES_PRICE')
plt.xticks(rotation=45)
plt.show()

Checking Relationship between BUILDTYPE and SALES_PRICE

In [ ]:
buildtype_order=df.groupby('BUILDTYPE')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['BUILDTYPE']

In [ ]:
sns.barplot(x='BUILDTYPE',y='SALES_PRICE',data=df,order=buildtype_order)

COMMERCIAL buildtypes more expensive compared to house and other

In [ ]:
df.groupby('BUILDTYPE')['SALES_PRICE'].mean().plot()
plt.title('BUILDTYPE vs SALES_PRICE')
plt.xlabel('BUILDTYPE')
plt.ylabel('SALES_PRICE')
plt.xticks(rotation=45)
plt.show()

Checking Relationship between UTILITY_AVAIL and SALES_PRICE

In [ ]:
utility_avail_order=df.groupby('UTILITY_AVAIL')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['UTILITY_AVAIL']

In [ ]:
sns.barplot(x='UTILITY_AVAIL',y='SALES_PRICE',data=df)

In [ ]:
df.groupby('UTILITY_AVAIL')['SALES_PRICE'].mean().plot()
plt.title('UTILITY_AVAIL vs SALES_PRICE')
plt.xlabel('UTILITY_AVAIL')
plt.ylabel('SALES_PRICE')
plt.xticks(rotation=45)
plt.show()

There is a good relationship between UTILITY_AVAIL and SALES_PRICE target.


Checking Relationship between STREET and SALES_PRICE

In [ ]:
street_order=df.groupby('STREET')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['STREET']

In [ ]:
sns.barplot(x='STREET',y='SALES_PRICE',data=df,order=street_order)

GRAVEL STREET houde are more expensive than other street

In [ ]:
df.groupby('STREET')['SALES_PRICE'].mean().plot()
plt.title('STREET vs SALES_PRICE')
plt.xlabel('STREET')
plt.ylabel('SALES_PRICE')
plt.xticks(rotation=45)
plt.show()

Checking Relationship between MZZONE and SALES_PRICE

In [ ]:
mzzone_order=df.groupby('MZZONE')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['MZZONE']

In [ ]:
sns.barplot(x='MZZONE',y='SALES_PRICE',data=df,order=mzzone_order)

MZZONE with the values RM,RL,RH have higher sales_price than other values

In [ ]:
df.groupby('MZZONE')['SALES_PRICE'].mean().plot()
plt.title('MZZONE vs SALES_PRICE') 

Checking Relationship between N_BEDROOM  and SALES_PRICE

In [ ]:
n_bedroom_order=df.groupby('N_BEDROOM')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['N_BEDROOM']

In [ ]:
sns.barplot(x='N_BEDROOM',y='SALES_PRICE',data=df,order=n_bedroom_order)

the increse in bedroom, sales price also increses

In [ ]:
df.groupby('N_BEDROOM')['SALES_PRICE'].mean().plot()
plt.title('N_BEDROOM vs SALES_PRICE')
plt.xlabel('N_BEDROOM')
plt.ylabel('SALES_PRICE')
plt.xticks(rotation=45)
plt.show()

Checking Relationship between N_BATHROOM  and SALES_PRICE

In [ ]:
sns.barplot(x='N_BATHROOM',y='SALES_PRICE',data=df)

THE increse in bathroom sales price also incerses

In [ ]:
df.groupby('N_BATHROOM')['SALES_PRICE'].mean().plot()
plt.title('N_BATHROOM vs SALES_PRICE')
plt.xlabel('N_BATHROOM')
plt.ylabel('SALES_PRICE')
plt.xticks(rotation=45)
plt.show()

Checking Relationship between N_ROOM  and SALES_PRICE

In [ ]:
sns.barplot(x='N_ROOM',y='SALES_PRICE',data=df)

The increse in number of rooms sales price also increses

In [ ]:
df.groupby('N_ROOM')['SALES_PRICE'].mean().plot()
plt.title('N_ROOM vs SALES_PRICE')
plt.xlabel('N_ROOM')
plt.ylabel('SALES_PRICE')
plt.xticks(rotation=45)
plt.show()

EDA for Numerical Columns

Checing linear relationship between the INT_SQFT and SALES_PRICE target columns

In [ ]:
sns.distplot(df['INT_SQFT'])

In [ ]:
sns.regplot(x='INT_SQFT',y='SALES_PRICE',data=df,scatter_kws={'color':'red'},line_kws={'color':'black'})

There are strong relationship between the INT_SQFT feature and SALES_PRICE target columns


Checking Relationship between DIST_MAINROAD  and SALES_PRICE

In [ ]:
sns.distplot(df['DIST_MAINROAD'])

In [ ]:
sns.regplot(x='DIST_MAINROAD',y='SALES_PRICE',data=df,scatter_kws={'color':'red'},line_kws={'color':'black'})

There is no relationship between the dist_mainroad and SALES_PRICE target columns
It is better to drop this feature

Checking Relationship betweenQS_ROOMS and SALES_PRICE





In [ ]:
sns.distplot(df['QS_ROOMS'])

In [ ]:
sns.regplot(x='QS_ROOMS',y='SALES_PRICE',data=df,scatter_kws={'color':'red'},line_kws={'color':'black'})

There is no relationship in between QS_ROOMS and SALES_PRICE target columns.

It is better to drop this feature

Checking Relationship between QS_BEDROOM  and SALES_PRICE

In [ ]:
sns.distplot(df['QS_BEDROOM'])

In [ ]:
sns.regplot(x='QS_BEDROOM',y='SALES_PRICE',data=df,scatter_kws={'color':'red'},line_kws={'color':'black'})

There is no relationship in between QS_BEDROOM and SALES_PRICE target columns.

It is better to drop this feature

Checking Relationship between QS_BATHROOM  and SALES_PRICE

In [ ]:
sns.distplot(df['QS_BATHROOM'])

In [ ]:
sns.regplot(x='QS_BATHROOM',y='SALES_PRICE',data=df,scatter_kws={'color':'red'},line_kws={'color':'black'})

There is no relationship in between QS_BATHROOM and SALES_PRICE target columns.

It is better to drop this feature

Checking Relationship between QS_OVERALL  and SALES_PRICE

In [ ]:
sns.distplot(df['QS_OVERALL'])

In [ ]:
sns.regplot(x='QS_OVERALL',y='SALES_PRICE',data=df,scatter_kws={'color':'red'},line_kws={'color':'black'})

There is no relationship in between QS_BATHROOM and SALES_PRICE target columns.

It is better to drop this feature

Checking Relationship between AGE and SALES_PRICE

In [ ]:
sns.regplot(x='AGE',y='SALES_PRICE',data=df,scatter_kws={'color':'red'},line_kws={'color':'black'})

There is a opposite relationship in between the AGE and SALES_PRICE target columns.
This feture will be usefull 

Insights From the Analasis

*  The distribution of the data in all the columns are normally distributed in some columns only the data is very slightly skewed.

*  INT_SQFT,N_BEDROOM,N_BATHROOM,N_ROOMS FEATUREShaving strong relationship with SALES_PRICE target.These features are very important to train this model.

*  DIST_MAINROAD,QS_ROOMS,QS_BEDROOM,QS_BATHROOMS,QS_OVERALL has no relationship with SALES_PRICE target,so,we will dropall these features






In [ ]:
df.drop(['QS_OVERALL','QS_ROOMS','QS_BATHROOM','QS_BEDROOM','DIST_MAINROAD','DATE_BUILD','DATE_SALE','SALE_COND'],axis=1,inplace=True)

In [ ]:
df.head()

Encoding the data

Label Encoding for binary class data


In [ ]:
df.PARK_FACIL=df.PARK_FACIL.map({'Yes':1,'No':0})

One-Hot Encoding for Multi-Class Nominal data

In [ ]:
df=pd.get_dummies(df,columns=["BUILDTYPE","AREA","UTILITY_AVAIL","STREET","MZZONE"])
df.head()

In [ ]:
df.dtypes

In [ ]:
df['SALES_PRICE']=df['SALES_PRICE'].astype(int)

In [ ]:
df.head()

Linear Reggresion Model

In [ ]:
x=df.drop('SALES_PRICE',axis=1).values
y=df['SALES_PRICE'].values

Splitting

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

Scalling

In [ ]:
scalar=StandardScaler()
scalar.fit(x_train,y_train)
x_train_scaled=scalar.transform(x_train)
x_test_scaled=scalar.transform(x_test)

Model Fitting

In [ ]:
regressor = LinearRegression()
regressor.fit(x_train_scaled,y_train)

Predicting Values

In [ ]:
y_pred= regressor.predict(x_test_scaled)
y_pred

Evaluvation Metric-R2 Value

In [ ]:
print('R2-SCORE:',metrics.r2_score(y_test,y_pred))

K Nearest Neighbbhour Algorithm(kNN)

hyper parameter tunning,fitting the model

In [ ]:
for i in [1,2,3,4,5,6,7,8,9,10,20,30]:
  knn=KNeighborsRegressor(i)
  knn.fit(x_train_scaled,y_train)
  print("k value :",i,"train score :",knn.score(x_train_scaled,y_train))

Desicion Tree Regressor Algorithm

Fitting the model

In [ ]:
dt=DecisionTreeRegressor()
dt.fit(x_train_scaled,y_train)


Predicted values

In [ ]:
y_pred=dt.predict(x_test_scaled)
y_pred

Evaluation metric-R2 Score

In [ ]:
print('R2-SCORE:',r2_score(y_test,y_pred))

Random forest Reggressor Algorithm

Fitting the Model 

In [ ]:
rf=RandomForestRegressor(n_estimators=10)
rf.fit(x_train_scaled, y_train)

predcting values

In [ ]:
y_pred=rf.predict(x_test_scaled)
y_pred

Evaluvation Metric-R2 Score

In [ ]:
print('R2-SCORE:',r2_score(y_test,y_pred))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
rf=RandomForestRegressor(n_estimators=100,max_depth=4,max_features='sqrt')
rf.fit(x_train_scaled,y_train)
y_pred=rf.predict(x_test_scaled)

In [ ]:
print('R2- SCORE:',r2_score(y_test,y_pred))

XG Boost Algorithm

tunning based on the learning rate,fitting the model

In [ ]:
for lr in [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.2,0.5,0.7,1]:
  model=xgb.XGBRegressor(learning_rate=lr,n_estimators=100,verbosity=0)
  model.fit(x_train_scaled,y_train)
  model.score(x_test_scaled,y_test)
  print("Learning rate :",lr,"Train score:",model.score(x_train_scaled,y_train),"cross-Val score :",np.mean(cross_val_score(model,x_train_scaled,y_train)))

Evaluvation Metric R2-Score

In [ ]:
model=xgb.XGBRegressor(learning_rate=0.7,n_estimators=100)
model.fit(x_train_scaled,y_train)
model.score(x_test_scaled,y_test)

R2 Score of all Models

Linear REgression-0.96
K Nearest Neighbours-0.97
Random Regressor Algorithm-0.98
XG Boost algorithm-0.99